In [1]:
!uv pip install edgartools ipywidgets

Audited 2 packages in 137ms


## Download the data


In [9]:
from os import makedirs

import pandas as pd
from edgar import Company, set_identity

In [10]:
set_identity("Noel Jacob noeljacob91@gmail.com")
df = pd.read_csv("./mining.csv")

In [11]:
ticker_list: list[str] = df["Symbol"].tolist()
form_list = ["10-K", "10-Q", "8-K", "6-K", "DEF 14A"]

In [ ]:
# topic_df = pd.DataFrame(columns=["Ticker", "Form", "Date", "Topics"])
for ticker in ticker_list:
    try:
        company = Company(ticker)
        filings = company.get_filings(form=form_list)
        for filing in filings:
            form = filing.form
            date = filing.filing_date

            # filing_obj = filing.data_object()
            # if filing_obj is not None and len(filing_obj.items) > 0:
            #     topics = ",".join(filing_obj.items)
            #     print(f'so"{topics}"ok')
            #     topic_df.loc[len(topic_df)] = [ticker, form, date, topics]

            md = filing.markdown()
            parent_dir = f"./edgartools-data-new/{ticker}/{form}"
            makedirs(parent_dir, exist_ok=True)
            with open(f"{parent_dir}/{date}.md", mode="w+") as f:
                f.write(md)
    except Exception as e:
        print(f"Error {ticker}: {e}")

# topic_df.to_csv("./topics.csv", index=False)

No XBRL attachments found in filing Filing(company='ALLIANCE RESOURCE PARTNERS LP', cik=1086600, form='DEF 14A', filing_date='2022-10-12', accession_no='0001558370-22-014952')
No XBRL attachments found in filing Filing(company='ALLIANCE RESOURCE PARTNERS LP', cik=1086600, form='DEF 14A', filing_date='2009-09-25', accession_no='0001193125-09-198009')
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
Subheader 'COMPANY DATA' not found in header ']		IRS NUMBER'
No XBRL attachments found in filing Filing(company='American Bitcoin Corp.', cik=1755953, form='DEF 14A', filing_date='2025-04-21', accession_no='0001213900-25-033808')
No XBRL attachments found in filing Filing(company='American Bitcoin Corp.', cik=1755953, form='DEF 14A', filing_

## Remove older than 2009


In [1]:
import os
from datetime import datetime

In [2]:
def delete_old_files(root_dir, cutoff_year=2009):
    cutoff_date = datetime(cutoff_year, 1, 1)
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".md"):
                try:
                    file_date = datetime.strptime(filename[:-3], "%Y-%m-%d")
                    if file_date < cutoff_date:
                        file_path = os.path.join(dirpath, filename)
                        os.remove(file_path)
                        print(f"Deleted: {file_path}")
                except ValueError:
                    pass

In [3]:
delete_old_files("./edgartools-data")

Deleted: ./edgartools-data/ARLP/10-Q/2008-11-07.md
Deleted: ./edgartools-data/ARLP/10-Q/2008-08-08.md
Deleted: ./edgartools-data/ARLP/10-Q/2008-05-12.md
Deleted: ./edgartools-data/ARLP/10-Q/2007-11-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2007-08-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2007-05-10.md
Deleted: ./edgartools-data/ARLP/10-Q/2006-11-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2006-08-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2006-05-10.md
Deleted: ./edgartools-data/ARLP/10-Q/2005-11-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2005-08-15.md
Deleted: ./edgartools-data/ARLP/10-Q/2005-05-10.md
Deleted: ./edgartools-data/ARLP/10-Q/2004-11-08.md
Deleted: ./edgartools-data/ARLP/10-Q/2004-08-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2004-05-07.md
Deleted: ./edgartools-data/ARLP/10-Q/2003-11-10.md
Deleted: ./edgartools-data/ARLP/10-Q/2003-08-11.md
Deleted: ./edgartools-data/ARLP/10-Q/2003-05-09.md
Deleted: ./edgartools-data/ARLP/10-Q/2002-11-14.md
Deleted: ./edgartools-data/ARLP

## Filter using keywords


In [4]:
ENERGY_STEMS = [
    # Power & Energy basics
    r"electr",  # electric, electrical, electricity
    r"power",
    r"energ",
    r"watt",  # watt, watts, wattage, kilowatt, megawatt, gigawatt
    r"volt",  # volt, voltage, kilovolt
    r"amp",  # amp, amps, ampere, amperage
    r"joule",
    r"btu",
    r"therm",
    # Units
    r"kwh",
    r"mwh",
    r"gwh",
    r"twh",
    r"btu",
    # Mining specific
    r"hash",  # hash, hashing, hashrate
    r" asic",
    # r"mine",  # miner, miners, mining
    r"s/s"
    r"h/s",
    r"/h",
    r"/m"
    r"/h",
    r"/kh",
    r"/mh"
    r"/gh",
    r"/th",
    # Generation & Sources
    r"generat",  # generate, generation, generator
    r"solar",
    r"wind",
    r"hydro",
    r"nuclear",
    r"coal",
    r"geotherm",
    r"bio",
    r"flare",  # flared gas
    r"renewab",  # renewable, renewables
    r"fossil",
    r"fuel",
    # Infrastructure
    r"grid",
    r"substation",
    r"transformer",
    r"transmiss",  # transmission
    r"data center",
    r"data-center"
    r"datacenter",
    r"facilit",  # facility, facilities
    r"interconnect",
    # Cooling & Efficiency
    r"cool",  # cooling, cooled, cooler
    r"hvac",
    r"pue",
    r"immersion",
    r"heat",
    # Costs & Contracts
    r"utilit",  # utility, utilities
    r"tariff",
    r" ppa",
    r"curtail",  # curtail, curtailment
    r"wholesale",
    # r"rate",
    # Environmental
    r" epa",
    r"carbon",
    r"co2",
    r"emission",
    r"greenhouse",
    r"ghg",
    r"sustainab",  # sustainable, sustainability
    r"esg",
    r"footprint",
    r"net zero",
    r"net-zero",
    r"scope 1",
    r"scope 2",
    r"scope 3",
    r"natural",
    r" nature",
    r"environment",
    # Storage & Backup
    r"battery",
    r"storage",
    r"ups",
    r"diesel",
    r"backup",
    # Regulatory
    r"ercot",
    r"ferc",
    r"pjm",
    r"nerc",
    r"iso",
    r"rto",
    # Consumption terms
    r"consum",  # consume, consumption, consumer
    r"usage",
    r"utiliz",  # utilize, utilization
    r"efficien",  # efficient, efficiency
    r"capacity",
    r"load",
    r"demand",
    r"supply",
    r"procure",  # procure, procurement
]

In [5]:
import re

ENERGY_PATTERN = re.compile("|".join(ENERGY_STEMS), re.IGNORECASE)

In [6]:
from pathlib import Path


def filter_energy_files(root_dir, dry_run=True):
    """Delete files without energy mentions. Set dry_run=False to actually delete."""
    deleted = []
    kept = []

    for filepath in Path(root_dir).rglob("*.md"):
        with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()

        if ENERGY_PATTERN.search(content):
            kept.append(str(filepath))
        else:
            print(f"Deleted: {filepath}")
            deleted.append(str(filepath))
            if not dry_run:
                os.remove(filepath)

    print(f"Kept: {len(kept)} files")
    print(f"{'Would delete' if dry_run else 'Deleted'}: {len(deleted)} files")

    return kept, deleted


kept, to_delete = filter_energy_files("./edgartools-data", dry_run=True)

Deleted: edgartools-data/ABTS/6-K/2025-10-24.md
Deleted: edgartools-data/ABTS/6-K/2025-10-10.md
Deleted: edgartools-data/ABTS/6-K/2025-08-13.md
Deleted: edgartools-data/ABTS/6-K/2025-05-29.md
Deleted: edgartools-data/ABTS/6-K/2025-03-25.md
Deleted: edgartools-data/ABTS/6-K/2024-11-25.md
Deleted: edgartools-data/ABTS/6-K/2024-11-06.md
Deleted: edgartools-data/ABTS/6-K/2024-10-30.md
Deleted: edgartools-data/ABTS/6-K/2024-10-15.md
Deleted: edgartools-data/ABTS/6-K/2024-10-11.md
Deleted: edgartools-data/ABTS/6-K/2024-08-16.md
Deleted: edgartools-data/ABTS/6-K/2024-08-06.md
Deleted: edgartools-data/ABTS/6-K/2024-05-22.md
Deleted: edgartools-data/ABTS/6-K/2024-04-15.md
Deleted: edgartools-data/ABTS/6-K/2024-01-30.md
Deleted: edgartools-data/ABTS/6-K/2024-01-11.md
Deleted: edgartools-data/ABTS/6-K/2023-11-16.md
Deleted: edgartools-data/ABTS/6-K/2023-11-08.md
Deleted: edgartools-data/ABTS/6-K/2023-10-18.md
Deleted: edgartools-data/ABTS/6-K/2023-08-15.md
Deleted: edgartools-data/ABTS/6-K/2023-0

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

import pandas as pd


def process_file(filepath: Path) -> list[dict]:
    """Process a single file and return matching records."""
    records = []
    try:
        with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        ticker = filepath.parts[1]
        form = filepath.parts[2]
        date = filepath.stem

        for line_num, line in enumerate(lines, start=1):
            matches = ENERGY_PATTERN.findall(line)
            if matches:
                records.append(
                    {
                        "ticker": ticker,
                        "form": form,
                        "date": date,
                        "filepath": str(filepath),
                        "line_num": line_num,
                        "matched_terms": ", ".join(set(m.lower() for m in matches)),
                        "line_text": line.strip()[:500],
                    }
                )
    except Exception as e:
        print(f"Error processing {filepath}: {e}")

    return records


def extract_matching_sentences_parallel(root_dir, max_workers=8):
    """Extract sentences with energy matches using parallel processing."""
    files = list(Path(root_dir).rglob("*.md"))
    all_records = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_file, f): f for f in files}

        for i, future in enumerate(as_completed(futures)):
            records = future.result()
            all_records.extend(records)
            if (i + 1) % 400 == 0:
                print(f"Processed {i + 1}/{len(files)} files...")

    return pd.DataFrame(all_records)


# Run parallel extraction
matches_df = extract_matching_sentences_parallel("./edgartools-data", max_workers=16)
print(f"Found {len(matches_df)} matching lines")
print(f"Unique tickers: {matches_df['ticker'].nunique()}")
print(f"Unique terms matched:\n{matches_df['matched_terms'].value_counts().head(20)}")

matches_df.to_csv("./energy_matches_debug.csv", index=False)
print("Saved to energy_matches_debug.csv")

Processed 200/2878 files...
Processed 400/2878 files...
Processed 600/2878 files...
Processed 800/2878 files...
Processed 1000/2878 files...
Processed 1200/2878 files...
Processed 1400/2878 files...
Processed 1600/2878 files...
Processed 1800/2878 files...
Processed 2000/2878 files...
Processed 2200/2878 files...
Processed 2400/2878 files...
Processed 2600/2878 files...
Processed 2800/2878 files...
Found 48004 matching lines
Unique tickers: 31
Unique terms matched:
matched_terms
coal             6317
power            5151
facilit          3060
generat          1924
electr           1754
demand           1444
iso              1393
consum           1121
environment      1103
amp              1077
hash             1039
utiliz            984
energ             756
supply            662
 nature           630
immersion         591
efficien          481
supply, power     481
data center       446
cool              431
Name: count, dtype: int64
Saved to energy_matches_debug.csv


In [8]:
matches_df["ticker"].value_counts().head(20)

ticker
GPUS    18367
ARLP    14365
BMNR     6244
ABTC     2096
ARBK     1399
CLSK      684
CORZ      561
SLNH      532
WULF      471
RIOT      467
GREE      441
MARA      378
SAIH      351
HUT       268
BITF      179
BTOG      171
CIFR      170
DGXX      161
IREN      148
ANY       111
Name: count, dtype: int64